In [ ]:
%matplotlib inline

In [ ]:

!/usr/bin/env python3

This script tune DEEP_RF Moldel for Charged Particles and extract Weight Matrix #
to Correct the loglikelihoods of Belle II Detectors                             #
								                  #
 Writtien by Alì Bavarchee                                                      #
##################################################################################

In [ ]:
import basf2 as b2

import numpy as np
import pandas as pd
from argparse import ArgumentParser
import h5py
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
#from .binary_tree import TorchDecisionTreeClassifier, TorchDecisionTreeRegressor
#from .utils import sample_vectors, sample_dimensions

import numpy as np

from os import makedirs
from os.path import join, dirname
from tqdm.auto import tqdm

import torch.nn.functional as F
import numpy as np
import pandas as pd
import itertools
from mpl_toolkits.mplot3d import Axes3D
from scipy import linalg as la
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import glob
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from os import makedirs
from os.path import join, dirname
from tqdm.auto import tqdm
import time
import pickle
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.datasets import make_regression

from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
# _make_const_lists gives some cte values ; Particles and their corrispond PDG_code, Detectors
def _make_const_lists():
    """Moving this code into a function to avoid a top-level ROOT import."""
    import ROOT.Belle2

    PARTICLES, PDG_CODES = [], []
    for i in range(len(ROOT.Belle2.Const.chargedStableSet)):
        particle = ROOT.Belle2.Const.chargedStableSet.at(i)
        name = (particle.__repr__()[7:-1]
                .replace("-", "")
                .replace("+", "")
                .replace("euteron", ""))
        PARTICLES.append(name)
        PDG_CODES.append(particle.getPDGCode())
    # PARTICLES = ["e", "mu", "pi", "K", "p", "d"]
    # PDG_CODES = [11, 13, 211, 321, 2212, 1000010020]

    DETECTORS = []
    for det in ROOT.Belle2.Const.PIDDetectors.set():
        DETECTORS.append(ROOT.Belle2.Const.parseDetectors(det))
    # DETECTORS = ["SVD", "CDC", "TOP", "ARICH", "ECL", "KLM"]

    return PARTICLES, PDG_CODES, DETECTORS

#This is a common pytorch data loader which loads data and splits them to train and test(val)
def load_training_data(directory, p_lims=None, theta_lims=None, device=None):
    """Loads training and validation data within the given momentum and theta
    limits (if given).

    Args:
        directory (str): Directory containing the train and validation sets.
        p_lims (tuple(float), optional): Minimum and maximum momentum. Defaults
            to None.
        theta_lims (tuple(float), optional): Minimum and maximum theta in
            degrees. Defaults to None.
        device (torch.device, optional): Device to move the data onto. Defaults
            to None.

    Returns:
        torch.Tensor: Training log-likelihood data.
        torch.Tensor: Training labels.
        torch.Tensor: Validation log-likelihood data.
        torch.Tensor: Validation labels.
    """
    p_lo, p_hi = p_lims if p_lims is not None else (-np.inf, +np.inf)
    t_lo, t_hi = theta_lims if theta_lims is not None else (-np.inf, +np.inf)
    t_lo, t_hi = np.radians(t_lo), np.radians(t_hi)

    def _load(filename):
        data = np.load(filename)
        X, y, p, t = data["X"], data["y"], data["p"], data["theta"]
        mask = np.logical_and.reduce([p >= p_lo, p <= p_hi, t >= t_lo, t <= t_hi])
        X = torch.tensor(X[mask]).to(device=device, dtype=torch.float)
        y = torch.tensor(y[mask]).to(device=device, dtype=torch.long)
        return X, y

    X_tr, y_tr = _load(join(directory, "train.npz"))
    X_va, y_va = _load(join(directory, "val.npz"))
    return X_tr, y_tr, X_va, y_va

data_folder = './data/slim_dstar'
df = load_training_data(data_folder)

X_tr, y_tr, X_va, y_va = load_training_data(data_folder)

#mask pdg code of pion and kaon (target sets) to 1 and -1

#Yy_tr = torch.where(y_tr==2,torch.tensor(1),y_tr)
#Yy_tr = torch.where(y_tr==3,torch.tensor(-1),y_tr)

#Yy_va = torch.where(y_va==2,torch.tensor(1),y_va)
#Yy_va = torch.where(y_va==3,torch.tensor(-1),y_va)






# INPUTS:
X_train= X_tr
y_train=y_tr  #.view(-1)
X_val= X_va
y_val= y_va   #.view(-1)


In [ ]:
#MixedRegressor UPDATED*


class MixedRegressor(nn.Module):
    def __init__(self, n_class, n_detector, const_init=None, rf_params=None, pretrained_model=None):
        super(MixedRegressor, self).__init__()  # Call the base class's constructor
        
        # Initialize attributes
        self.n_class = n_class
        self.n_detector = n_detector
        self.const_init = const_init
        self.rf_params = rf_params
        
        self.rf_net = None  # Initialize rf_net

        if pretrained_model is not None:
            self.neural_net = pretrained_model
            self._initialize_pretrained_layers(n_class, n_detector)
        else:
            self.neural_net = models.resnet18(pretrained=False)
            self._initialize_new_layers(n_class, n_detector)
        self.fcs = nn.ModuleList([nn.Linear(n_detector, 1) for _ in range(n_class)])  # Initialize fcs

        if self.const_init is not None:
            self.const_layer = nn.Linear(self.n_detector, self.n_class)
            self.const_layer.bias.data = torch.tensor([self.const_init] * self.n_class, dtype=torch.float32)
        
        if self.rf_params is not None:
            self.random_forest = RandomForestRegressor(
                n_estimators=self.rf_params['n_estimators'],
                max_depth=self.rf_params['max_depth']
            )
            

    def _initialize_new_layers(self, n_class, n_detector):
        num_features = self.neural_net.fc.in_features
        self.neural_net.fc = nn.Sequential(
            nn.Linear(num_features, n_class),  # Adjust output size for regression
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(n_class, n_detector * n_class)  # Output for each class-detector pair
        )

    def forward(self, x):
        n = self.n_detector
        outs = [self.fcs[i](x[:, i * n: (i + 1) * n]) for i in range(self.n_class)]
        out = torch.cat(outs, dim=1)
        return out

    def const_init(self, const):
        with torch.no_grad():
            for fc in self.fcs:
                fc.weight.fill_(const)

    def random_init(self, mean=1.0, std=0.5):
        with torch.no_grad():
            for fc in self.fcs:
                fc.weight.fill_(0)
                fc.weight.add_(torch.normal(mean=mean, std=std, size=fc.weight.size()))

    def kill_unused(self, only):
        if only is not None:
            for i, pdg in enumerate(PDG_CODES):
                if pdg in only:
                    continue
                self.fcs[i].weight.requires_grad = False
                self.fcs[i].weight.fill_(1)

    def train_neural(self, X_train, y_train, device='cpu', epochs=10, use_tqdm=True):
        self.to(device=device)
        opt = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=5e-4)
        iterator = range(epochs)
        neural_losses = []
        rf_losses = []
        if use_tqdm:
            iterator = tqdm(iterator)

        for epoch in iterator:
            self.train()
            opt.zero_grad()
            out = self(X_train)
            y_train_reshaped = y_train.float().reshape(-1, 1)
            neural_loss_value = self.neural_loss(out, y_train_reshaped)
            neural_losses.append(neural_loss_value.item())
            # Perform the backward pass
            neural_loss_value.backward()
            #loss.backward()
            opt.step()
            # Calculate and record RF loss
            # Record neural loss value
            #rf_loss_value = self.rf_net.loss(X_train, y_train).item()
            #rf_losses.append(rf_loss_value)
            rf_losses.append(0)  # Placeholder since random forest doesn't use the same loss
            

    def neural_loss(self, output, target):
        loss = nn.MSELoss()
        return loss(output, target)

    def train_rf(self, X_train, y_train):
        if self.rf_net is None:
            self.rf_net = RandomForestRegressor(**self.rf_params)  # Initialize rf_net if not already initialized
        self.rf_net.fit(X_train, y_train)  # Train rf_net

    def predict(self, X_test):
        neural_predictions = self.forward(X_test)
        rf_predictions = self.rf_net.predict(X_test)
        
        # Detach tensors from the computation graph and then convert to NumPy arrays
        neural_predictions_np = neural_predictions.detach().numpy()
        rf_predictions_np = rf_predictions.reshape(-1, 1)  # Reshape to match neural_predictions
        
        combined_predictions = (neural_predictions_np + rf_predictions_np) / 2.0
        return combined_predictions

    def score(self, X_test, y_test):
        predictions = self.predict(X_test)
        #mse = mean_squared_error(y_test, predictions)
        # Repeat each target value for each output unit
        y_true_reshaped = np.repeat(y_test[:, np.newaxis], 6, axis=1)
        mse_per_output = mean_squared_error(y_true_reshaped, predictions, multioutput='raw_values')
        avg_mse = mse_per_output.mean()
        return avg_mse

    def get_weights(self, to_numpy=False, device=None):
        neural_weights = None
        if hasattr(self, 'neural_net'):
            neural_weights = [param.detach().cpu().numpy() for param in self.neural_net.parameters()]
   
        rf_weights = self.rf_net.feature_importances_

        if neural_weights is None:
            combined_weights_matrix = rf_weights
        else:
            num_neural_weights = len(neural_weights)
            num_rf_weights = len(rf_weights)
            combined_weights_matrix = np.zeros((num_neural_weights, num_rf_weights))

            for i in range(num_neural_weights):
                for j in range(num_rf_weights):
                    if neural_weights[i].shape == rf_weights[j].shape:
                        combined_weights_matrix[i, j] = neural_weights[i] + rf_weights[j]
                    else:
                        # Handle the case where shapes are not compatible
                        # You might need to reshape, slice, or modify the weights here
                        combined_weights_matrix[i, j] = rf_weights[j]  # Placeholder value, adjust as needed

        if to_numpy:
            return combined_weights_matrix
        else:
            return torch.tensor(combined_weights_matrix, device=device)
    
    def save_model(self, file_path):
        torch.save(self.neural_net.state_dict(), file_path)
        
    def load_model(self, file_path):
        self.neural_net.load_state_dict(torch.load(file_path))


In [ ]:
#####
#INI# TTTTex
#####




# Define the hyperparameter search space
hyperparameters = {
    'lr': [0.0005],
    'hidden_units': [1],
    'dropout': [0.0],
    'rf_params': {
        'n_estimators': [200],
        'max_depth': [100],
    }
}

# Initialize variables to store results
mse_grid = np.zeros((len(hyperparameters['lr']), len(hyperparameters['hidden_units']),
                     len(hyperparameters['dropout']), len(hyperparameters['rf_params']['n_estimators']),
                     len(hyperparameters['rf_params']['max_depth'])))

# Initialize variables to store ROC curves
roc_curves = []
# Initialize variables to store best model and its performance
fpr = dict()
tpr = dict()
roc_auc = dict()
best_mse = float('inf')
best_auc = 0.0
best_model = None
best_weights = {}
best_hyperparameters = {}

# Initialize a subplot with multiple axes
fig = make_subplots(rows=len(hyperparameters['hidden_units']), cols=len(hyperparameters['lr']),
                    subplot_titles=[f"Hidden Units: {hu}, Learning Rate: {lr}" for hu in hyperparameters['hidden_units'] for lr in hyperparameters['lr']])

# Loop through all combinations of hyperparameters
for i, lr in enumerate(hyperparameters['lr']):
    for j, hidden_units in enumerate(hyperparameters['hidden_units']):
        # Initialize variables to store ROC curves
        roc_curves = []
        
        for k, dropout in enumerate(hyperparameters['dropout']):
            for m, n_estimators in enumerate(hyperparameters['rf_params']['n_estimators']):
                for n, max_depth in enumerate(hyperparameters['rf_params']['max_depth']):
                    # Instantiate the MixedRegressor with current hyperparameters
                    model = MixedRegressor(
                        n_class=6,
                        n_detector=6,
                        const_init=1,
                        rf_params={
                            'n_estimators': n_estimators,
                            'max_depth': max_depth
                        }, pretrained_model = None
                    )

                    # Train the model
                    model.train_neural(X_train, y_train, device='cpu', epochs=100)
                    model.train_rf(X_train, y_train)

                    # Get the combined predictions
                    combined_predictions = model.predict(X_val)
                    
                    # Calculate the model's performance on validation data
                    mse = model.score(X_val, y_val)
                    mse_grid[i, j, k, m, n] = mse
                    

                    # Update best model if the current model is better
                    if mse < best_mse:
                        best_mse = mse
                        best_model = model
                        best_hyperparameters = {
                            'lr': lr,
                            'hidden_units': hidden_units,
                            'dropout': dropout,
                            'n_estimators': n_estimators,
                            'max_depth': max_depth
                        }

                    # Compute ROC curves and AUC for each class
                    for class_idx in range(6):
                        fpr, tpr, _ = roc_curve(y_val, combined_predictions[:, class_idx], pos_label=class_idx)
                        roc_auc = auc(fpr, tpr)
                        roc_curves.append((fpr, tpr, roc_auc))

            # Plot ROC curves for each class
            for class_idx in range(6):
                fpr, tpr, roc_auc = roc_curves[class_idx]
                trace = go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Class {class_idx} (AUC = {roc_auc:.2f})')
                fig.add_trace(trace, row=j+1, col=i+1)
                
                
# Visualize the results using a heatmap
best_indices = np.unravel_index(np.argmin(mse_grid), mse_grid.shape)
best_mse = mse_grid[best_indices]
best_hyperparameters = {
    'lr': hyperparameters['lr'][best_indices[0]],
    'hidden_units': hyperparameters['hidden_units'][best_indices[1]],
    'dropout': hyperparameters['dropout'][best_indices[2]],
    'n_estimators': hyperparameters['rf_params']['n_estimators'][best_indices[3]],
    'max_depth': hyperparameters['rf_params']['max_depth'][best_indices[4]],
}

# Update subplot layout
for row in range(1, len(hyperparameters['hidden_units'])+1):
    for col in range(1, len(hyperparameters['lr'])+1):
        fig.update_xaxes(title_text='False Positive Rate', row=row, col=col)
        fig.update_yaxes(title_text='True Positive Rate', row=row, col=col)

# Update main layout
fig.update_layout(
    title='ROC Curves for Different Hyperparameter Combinations',
    showlegend=False
)

# Plot ROC curves for each class
plt.figure(figsize=(10, 6))
for class_idx in range(6):
    fpr, tpr, roc_auc = roc_curves[class_idx]
    plt.plot(fpr, tpr, label=f'Class {class_idx} (AUC = {roc_auc:.2f})')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Each Class')
plt.legend()
plt.grid()
plt.savefig('FINX_ROC Curves_XIX06.png')
plt.show()

# Plot Hyperparam GS Results
fig.write_html('FINX_roc_curves_XIX06.html')

plt.figure(figsize=(10, 6))
plt.imshow(np.log(mse_grid.mean(axis=4)[0, 0]), cmap='viridis')  # Taking log for better visualization
plt.colorbar(label='Log Mean MSE')
plt.xticks(range(len(hyperparameters['hidden_units'])), hyperparameters['hidden_units'])
plt.yticks(range(len(hyperparameters['hidden_units'])), hyperparameters['hidden_units'])
plt.xlabel('Dropout')
plt.ylabel('Learning Rate')
plt.title('Hyperparam GS Results')
plt.savefig('FINX_Hyperparam GS Results_XIX06.png')
plt.show()



print("================================================")
print("Best Hyperparameters:", best_hyperparameters)
print("================================================")
print("Best Mean MSE:", best_mse)
print("================================================")
# Save the best model

best_model.save_model('FINX_best_DeepRF_XIX6.pth')

print("||||||||||||||||||||||")

# Extract the best model's weights
best_model_keyz = best_model.neural_net.state_dict()
kkeyz = [key for key in best_model_keyz.keys()]
best_weights = best_model.neural_net.state_dict()[kkeyz[len(kkeyz)-1]]
best_weights_reshaped = best_weights.reshape((6, 6))+1
print("best_weights:")
print(best_weights_reshaped)
np.save('FINX_best_weights_XIX06.npy', best_weights_reshaped)



print('__MISSION COMPELETED__')